In [2]:
import pandas as pd
import numpy as np

import gensim.downloader as api
from gensim.models import KeyedVectors
from gensim.models import word2vec

from sklearn.model_selection import train_test_split as split

import contractions

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv1D, Bidirectional, LSTM, Dense, Concatenate, Dropout, Embedding, GlobalMaxPooling1D
from tensorflow.keras.metrics import binary_crossentropy, categorical_crossentropy, Accuracy, Recall, Precision
from tensorflow.keras.initializers import Constant

In [3]:
X = pd.read_feather('train.feather').sample(1000000).reset_index(drop=True)
Y = X.pop('key')

X_train, X_test, y_train, y_test = split(X, Y, test_size=0.3)

def totensor(df):
    return tf.convert_to_tensor(np.array(df.map(lambda x: x.tolist()).values.tolist()))

X_train = totensor(X_train.squeeze())
X_test = totensor(X_test.squeeze())
y_train = tf.convert_to_tensor(y_train)
y_test = tf.convert_to_tensor(y_test)

In [4]:
n_filters=128
sent_len = 20

weights = api.load('glove-wiki-gigaword-200').vectors

inp = Input(shape=(sent_len,))

emb = Embedding(input_dim=weights.shape[0],
                output_dim=weights.shape[1],
                embeddings_initializer=Constant(weights),
                trainable=False)

conv2 = Conv1D(filters=n_filters,
              activation='relu',
              padding='same',
              kernel_size=2)

conv3 = Conv1D(filters=n_filters,
              activation='relu',
              padding='same',
              kernel_size=3)

conv5 = Conv1D(filters=n_filters,
              activation='relu',
              padding='same',
              kernel_size=5)

model2 = GlobalMaxPooling1D()(conv2(emb(inp)))
model3 = GlobalMaxPooling1D()(conv3(emb(inp)))
model5 = GlobalMaxPooling1D()(conv5(emb(inp)))

conc = Concatenate()([model2, model3, model5])
decode = Dropout(0.3)(Dense(256)(conc))
out = Dense(1, activation='sigmoid')(decode)

model = Model(inputs=inp, outputs=out)
model.compile(loss=binary_crossentropy, optimizer='adam', metrics=[Accuracy(), Recall()])

In [7]:
model.fit(X_train, y_train,
          epochs=10,
          batch_size=32,
          validation_data=(X_test, y_test),
          verbose=1)

Train on 700000 samples, validate on 300000 samples
Epoch 1/10
700000/700000 [==============================] - 166s 237us/sample - loss: 0.0091 - accuracy: 0.6251 - recall: 0.9980 - val_loss: 0.0830 - val_accuracy: 0.5469 - val_recall: 0.9930
Epoch 2/10
700000/700000 [==============================] - 167s 238us/sample - loss: 0.0088 - accuracy: 0.6872 - recall: 0.9982 - val_loss: 0.1183 - val_accuracy: 0.6849 - val_recall: 0.9934
Epoch 3/10
700000/700000 [==============================] - 166s 238us/sample - loss: 0.0086 - accuracy: 0.7056 - recall: 0.9984 - val_loss: 0.1840 - val_accuracy: 0.8476 - val_recall: 0.9920
Epoch 4/10
700000/700000 [==============================] - 166s 237us/sample - loss: 0.0082 - accuracy: 0.7429 - recall: 0.9984 - val_loss: 0.1506 - val_accuracy: 0.7825 - val_recall: 0.9902
Epoch 5/10
700000/700000 [==============================] - 166s 237us/sample - loss: 0.0086 - accuracy: 0.7654 - recall: 0.9985 - val_loss: 0.1250 - val_accuracy: 0.6978 - val_rec

In [9]:
model.evaluate(X_test, y_test, batch_size=64)

300000/300000 [==============================] - 24s 80us/sample - loss: 0.1996 - accuracy: 0.8468 - recall: 0.9901


[0.1996402103018937, 0.84675336, 0.9901246]